
# Markowitz Demo — Max Sharpe vs Min Variance vs Equal Weight

Ce notebook charge des prix (AAPL, MSFT, NVDA, TSLA), calcule les rendements, 
puis compare **trois portefeuilles** :
- **Max Sharpe**
- **Min Variance**
- **Equal Weight (25% chacun)**

Il trace également la **frontière efficiente** et sauvegarde un **résumé CSV**.


In [ ]:

# --- Imports & configuration ---
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.optimize import minimize

DATA_PATH = Path("data/prices_yf.csv")  # colonnes attendues: Date, AAPL, MSFT, NVDA, TSLA
RISK_FREE = 0.02  # 2% annualisé (adapter)
ASSETS = ["AAPL", "MSFT", "NVDA", "TSLA"]


In [ ]:

# --- Chargement des prix ---
if not DATA_PATH.exists():
    raise FileNotFoundError(
        f"Fichier introuvable: {DATA_PATH}\n"
        "Assurez-vous d'avoir exécuté src/download_data.py, ou modifiez DATA_PATH.\n"
        "Format attendu: CSV avec colonnes 'Date,AAPL,MSFT,NVDA,TSLA'."
    )

prices = pd.read_csv(DATA_PATH, parse_dates=["Date"]).set_index("Date").sort_index()
prices = prices[ASSETS]
display(prices.tail())


In [ ]:

# --- Rendements & statistiques annualisées ---
rets = prices.pct_change().dropna()
mu_daily = rets.mean()
cov_daily = rets.cov()

# Annualisation (252 jours de bourse)
mu_ann = mu_daily * 252
cov_ann = cov_daily * 252

print("Rendements annualisés (mu):")
display(mu_ann.to_frame("mu").T)

print("Matrice de covariance annualisée:")
display(cov_ann)


In [ ]:

# --- Fonctions utilitaires ---
def portfolio_perf(w, mu, cov, rf=0.0):
    """
    Calcule rendement annualisé, volatilité annualisée et Sharpe pour des poids w.
    w: np.ndarray shape (n,)
    mu: pd.Series (mu annualisé)
    cov: pd.DataFrame (cov annualisée)
    rf: taux sans risque annualisé
    """
    w = np.asarray(w)
    exp_return = float(np.dot(w, mu.values))
    exp_vol = float(np.sqrt(np.dot(w, np.dot(cov.values, w))))
    sharpe = (exp_return - rf) / exp_vol if exp_vol > 0 else np.nan
    return exp_return, exp_vol, sharpe

def max_drawdown(series):
    """Max drawdown (retourne une valeur <= 0)."""
    cummax = series.cummax()
    dd = series / cummax - 1.0
    return float(dd.min())

def portfolio_path(w, prices_df):
    """
    Construit la valeur cumulée d un portefeuille buy&hold à partir des prix.
    Normalise à 1 au début.
    """
    w = np.asarray(w)
    norm_prices = prices_df / prices_df.iloc[0]
    return (norm_prices * w).sum(axis=1)

def summarize_portfolio(name, w, mu, cov, rf, prices_df):
    exp_ret, exp_vol, sharpe = portfolio_perf(w, mu, cov, rf)
    path = portfolio_path(w, prices_df)
    mdd = max_drawdown(path)
    out = {
        "portfolio": name,
        "exp_return": exp_ret,
        "exp_vol": exp_vol,
        "sharpe": sharpe,
        "max_drawdown": mdd,
    }
    for i, a in enumerate(prices_df.columns):
        out[f"weight_{a}"] = w[i]
    return out


In [ ]:

# --- Optimisations: Max Sharpe & Min Variance ---
n = len(ASSETS)
w0 = np.ones(n) / n
bounds = [(0.0, 1.0)] * n  # pas de vente à découvert
cons_sum = {"type": "eq", "fun": lambda w: np.sum(w) - 1.0}

def neg_sharpe(w, mu, cov, rf):
    er, ev, sh = portfolio_perf(w, mu, cov, rf)
    return -sh

res_max_sharpe = minimize(neg_sharpe, w0, args=(mu_ann, cov_ann, RISK_FREE),
                          method="SLSQP", bounds=bounds, constraints=[cons_sum])
w_ms = res_max_sharpe.x

def variance(w, cov):
    w = np.asarray(w)
    return float(np.dot(w, np.dot(cov.values, w)))

res_min_var = minimize(variance, w0, args=(cov_ann,),
                       method="SLSQP", bounds=bounds, constraints=[cons_sum])
w_mv = res_min_var.x

w_ew = np.ones(n) / n

print("Poids Max Sharpe:", np.round(w_ms, 4))
print("Poids Min Var   :", np.round(w_mv, 4))
print("Poids Equal Wgt :", np.round(w_ew, 4))


In [ ]:

# --- Résumés & export CSV ---
rows = []
rows.append(summarize_portfolio("max_sharpe", w_ms, mu_ann, cov_ann, RISK_FREE, prices))
rows.append(summarize_portfolio("min_variance", w_mv, mu_ann, cov_ann, RISK_FREE, prices))
rows.append(summarize_portfolio("equal_weight", w_ew, mu_ann, cov_ann, RISK_FREE, prices))

summary = pd.DataFrame(rows)
display(summary)

out_path = Path("data/markowitz_summary.csv")
out_path.parent.mkdir(parents=True, exist_ok=True)
summary.to_csv(out_path, index=False)
print(f"Résumé exporté -> {out_path.resolve()}")


In [ ]:

# --- Frontière efficiente ---
def solve_min_var_for_target_return(target_r, mu, cov):
    n = len(mu)
    w0 = np.ones(n) / n
    bounds = [(0.0, 1.0)] * n
    cons = [
        {"type": "eq", "fun": lambda w: np.sum(w) - 1.0},
        {"type": "eq", "fun": lambda w, tr=target_r: np.dot(w, mu.values) - tr},
    ]
    res = minimize(variance, w0, args=(cov,), method="SLSQP", bounds=bounds, constraints=cons)
    return res

target_grid = np.linspace(summary["exp_return"].min()*0.6,
                          summary["exp_return"].max()*1.1, 40)

ef_points = []
for tr in target_grid:
    res = solve_min_var_for_target_return(tr, mu_ann, cov_ann)
    if res.success:
        w = res.x
        r, v, s = portfolio_perf(w, mu_ann, cov_ann, RISK_FREE)
        ef_points.append((v, r))

ef = pd.DataFrame(ef_points, columns=["exp_vol", "exp_return"])
display(ef.head())


In [ ]:

# --- Graphique: frontière + portefeuilles ---
plt.figure(figsize=(7,5))
plt.scatter(ef["exp_vol"], ef["exp_return"], s=12, label="Efficient Frontier")
for name, w in [("Max Sharpe", w_ms), ("Min Variance", w_mv), ("Equal Weight", w_ew)]:
    r, v, s = portfolio_perf(w, mu_ann, cov_ann, RISK_FREE)
    plt.scatter([v], [r], s=60, marker="x", label=f"{name}")
plt.xlabel("Volatilité annualisée")
plt.ylabel("Rendement annualisé")
plt.legend()
plt.title("Frontière efficiente (long-only) + portefeuilles")
plt.show()


In [ ]:

# --- Graphiques: Poids ---
def plot_weights(w, title):
    plt.figure(figsize=(6,4))
    plt.bar(ASSETS, w)
    plt.ylim(0, 1)
    plt.title(title)
    plt.ylabel("Poids")
    plt.show()

plot_weights(w_ms, "Poids — Max Sharpe")
plot_weights(w_mv, "Poids — Min Variance")
plot_weights(w_ew, "Poids — Equal Weight (25%)")



### Annexe — Télécharger les données si besoin
Si vous n'avez pas `data/prices_yf.csv`, exécutez votre script `src/download_data.py` (recommandé).
Sinon, exemple *ad hoc* (à adapter) :

```
# !pip install yfinance
import yfinance as yf, pandas as pd
from pathlib import Path

tickers = ["AAPL", "MSFT", "NVDA", "TSLA"]
data = yf.download(tickers, start="2020-01-01", end="2025-01-01")
if isinstance(data.columns, pd.MultiIndex):
    if "Adj Close" in data.columns.levels[0]:
        data = data["Adj Close"]
    else:
        data = data["Close"]
data = data.dropna().reset_index()
data.columns = ["Date"] + list(data.columns[1:])
Path("data").mkdir(exist_ok=True)
data.to_csv("data/prices_yf.csv", index=False)
```
